# Synodal Bible & Mother

## Summary

In [ ]:
# Code summary of below

In [ ]:
!python --version

## Import

### Libraries

In [ ]:
from backend import *
import multiprocessing as mp
from multiprocessing import Pool, Process
import pickle
num_cores = int(mp.cpu_count()-2)
print(f"Number of cores : {num_cores}")

In [ ]:
%matplotlib inline

### Definitions

In [ ]:
texts = '../texts/fiction/utf8/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', \
             'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

In [ ]:
motherTokenDf = pd.read_pickle('./proc/MotherTokendf.pkl')
bibleTokenDf = pd.read_pickle('./proc/BibleTokenDf.pkl')

In [ ]:
# add stopword boolean, True if all attribute columns are null
motherTokenDf['stopword'] = ~motherTokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDfIdx = pd.Index(range(1, (motherTokenDf.shape[0]+1)), name='id')
motherTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

bibleTokenDf['stopword'] = ~bibleTokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDfIdx = pd.Index(range(1, (bibleTokenDf.shape[0]+1)), name='id')
bibleTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

## Bible and Mother Token Dataframes

In [ ]:
bibleTokenDf

In [ ]:
motherTokenDf#.set_index(['p_id','token_id'])

In [ ]:
lemmaDf = bibleTokenDf[['lemma']].loc[bibleTokenDf.stopword == False].join(motherTokenDf[['lemma']].loc[motherTokenDf.stopword == False], on='id', lsuffix='_bible', rsuffix='_mother')
lemmaDf

## Shared Lemmas

In [ ]:
bible_lemmas = bibleTokenDf.loc[bibleTokenDf.stopword == False].lemma
mother_lemmas = motherTokenDf.loc[motherTokenDf.stopword == False].lemma

In [ ]:
common_lemmas = []
common_lemmas_count = 0
for lemma in mother_lemmas.unique():
    #print(lemma)
    if lemma in bible_lemmas.unique():
        common_lemmas.append(lemma)
        common_lemmas_count += 1
        
print(f"Shared unique words between Mother and the Bible: "+str(common_lemmas_count)+", that's ~"+str(round(((common_lemmas_count/len(lemmaDf.lemma_mother.unique()))*100),2))+"%")

In [ ]:
def ListSplit(lst, numGroups, sort=True): 
    """Takes a list and a number and splits the 
    list into evenly divided n groups (as much as possible)"""
    # choose to sort list by ascending
    if sort: 
        lst.sort()
    # get length of list given to sort
    listLen = len(lst)
    # groupLen is the maximum number of items per group to allow for the most groups <= numGroups in a list (numGroups-1 OR numGroups)
    groupLen = (listLen//numGroups) + (listLen % numGroups > 0)
    # yield generator object of nested listed with length of numGroups 
    for i in range(0,len(lst), groupLen): 
        yield lst[i:i+groupLen]

def token_window(lemma_array, window_size):
    token_window_array = np.array([])
    token_window_array = np.array([np.array(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def token_window_mp(lemma_array, window_size):
    #token_window_array = np.array([])
    token_window_array = enumerate([list(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def get_sim_count(child_window_array, parent_window_array, order_matters=False):
    work_sim_count = 0
    work_sim = []
    for child_window in child_window_array:
        child_sim = {}
        #print(f"child window: {child_window}")
        child_window_sim_total_count = 0
        for parent_window in parent_window_array:
            #print(f"parent window: {parent_window}")
            child_window_sim_sub_count = 0
            for i in range(len(child_window)):
                if order_matters:
                    if child_window[1][i] == parent_window[1][i]:
                        child_window_sim_total_count+=1
                        child_window_sim_sub_count+=1
                else:
                    if child_window[1][i] in parent_window[1]:
                        #print(f"{child_window[i]} is in parent!")
                        child_window_sim_total_count+=1
                        child_window_sim_sub_count+=1
                    #else:
                        #print(f"{child_window[i]} is not in parent!")
            child_sim.update({parent_window[0]: child_window_sim_sub_count})
        #print(f"{child_window}: {child_window_sim_total_count}")
        work_sim.append({child_window_sim_total_count: child_sim})
    return work_sim

def dna_test_windows(child_lemmas, parent_lemmas, window_size=10):
    sim_counter = 0
    #sim_list = np.array([])
    parent_window_array = token_window(parent_lemmas, window_size)
    child_window_array = token_window(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    #print(sim_list)
    return sim_list

def dna_test_windows_mp(child_lemmas, parent_lemmas, window_size=10):
    #child_lemmas, parent_lemmas = cnp_lemmas[0], cnp_lemmas[1]
    #print(f"Child lemmas: {child_lemmas}")
    #print(f"Parent lemmas: {parent_lemmas[:10]}")
    sim_counter = 0
    sim_list = np.array([])
    parent_window_array = token_window_mp(parent_lemmas, window_size)
    child_window_array = token_window_mp(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    return sim_list

def window_count_sim_mp(child_window_array, parent_window_array, order_matters=False):
    sim_count_tuples = {}
    for (c_sub_array_id, c_sub_array_list) in child_window_array:
        c_sub_sim_count = 0
        for (p_sub_array_id, p_sub_array_list) in parent_window_array:
            c_p_sim_count = 0
            for i in range(len(c_sub_array_list)):
                if order_matters:
                    if c_sub_array_list[i] == p_sub_array_list[i]:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
                else:
                    if c_sub_array_list[i] in p_sub_array_list:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
            sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
    return sim_count_tuples

In [ ]:
def list_split(lst, numGroups, sort=True): 
    """Takes a list and a number and splits the 
    list into evenly divided n groups (as much as possible)"""
    # choose to sort list by ascending
    if sort: 
        lst.sort()
    # get length of list given to sort
    listLen = len(lst)
    # groupLen is the maximum number of items per group to allow for the most groups <= numGroups in a list (numGroups-1 OR numGroups)
    groupLen = (listLen//numGroups) + (listLen % numGroups > 0)
    # yield generator object of nested listed with length of numGroups 
    for i in range(0,len(lst), groupLen): 
        yield lst[i:i+groupLen]

def token_window_mp(lemma_array, window_size):
    #token_window_array = np.array([])
    token_window_array = enumerate([list(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def window_count_sim_mp(child_window_array, parent_window_array, order_matters=False):
    sim_count_tuples = {}
    for (c_sub_array_id, c_sub_array_list) in child_window_array:
        c_sub_sim_count = 0
        for (p_sub_array_id, p_sub_array_list) in parent_window_array:
            c_p_sim_count = 0
            for i in range(len(c_sub_array_list)):
                if order_matters:
                    if c_sub_array_list[i] == p_sub_array_list[i]:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
                else:
                    if c_sub_array_list[i] in p_sub_array_list:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
            sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
    return sim_count_tuples

def dna_test_windows_mp(child_lemmas, parent_lemmas, window_size=20):
    sim_list = {}
    parent_window_array = token_window_mp(parent_lemmas, window_size)
    child_window_array = token_window_mp(child_lemmas, window_size)
    sim_list = window_count_sim_mp(child_window_array, parent_window_array)
    return sim_list

In [ ]:
%%time
if __name__ == "__main__":
    pool = Pool(processes=num_cores)
    #workerAssignments = ((x, bible_lemmas) for x in ListSplit(mother_lemmas[:198], num_cores, False))
    starmap_list = list(zip(list_split(child_window_array, num_cores), list_split(parent_window_array, num_cores)))
    print(f"Splitting into {num_cores} groups")
    start_time = time.time()
    sim_tuples = pool.starmap(window_count_sim_mp, starmap_list, chunksize=2)
    print(f"Finished in {round(time.time()-start_time, 3)} seconds.")
    pool.close()
    pool.join()
    #print(sim_tuples)

In [ ]:
motherTokenDf.loc[motherTokenDf.lemma == "эммаус"]

In [ ]:
len(sim_tuples[0])

In [ ]:
sim_tuples

In [ ]:
total_sim_tuples = {}
for t in sim_tuples:
    total_sim_tuples.update(t)

In [ ]:
len(total_sim_tuples)

In [ ]:
with open('./proc/sim_tuples.pkl', 'wb') as file:
    pickle.dump(total_sim_tuples, file)

In [ ]:
child_window_array = list(token_window_mp(mother_lemmas, 20))
parent_window_array = list(token_window_mp(bible_lemmas, 20))

In [ ]:
[x for x in parent_window_array]

In [ ]:
child_window_array = list(token_window_mp(mother_lemmas, 20))
parent_window_array = list(token_window_mp(bible_lemmas, 20))
#[x for x in parent_window_array]
sim_count_tuples = window_count_sim(child_window_array, parent_window_array)

In [ ]:
sim_count_tuples.items()

In [ ]:
pd.DataFrame([x for tup in sim_count_tuples for x in tup], columns=['window_vocab', 'sim_count']).sort_values(by='sim_count', axis=0, ascending=False)

In [ ]:
import pickle
with open('./proc/sim_tuples.pkl', 'wb') as file:
    pickle.dump(sim_tuples, file)

In [ ]:
pd.DataFrame(sim_tuples, columns=['window_vocab', 'sim_count']).locsort_values(by='sim_count', axis=0, ascending=False)

How do Gor'kii's *Mother* and the Synodal Bible overlap? 
windows of 3-5 words with 1-2+ of overlap